### Pitching Analysis

Import the necessary statements and set the dataframes to show all the columns.

In [1]:
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

We import our dataframes for batting, pitching, and fielding statistics as well as our list of teams from the pickle file titled `Stats.pkl`.

In [2]:
with open('Stats.pkl', 'rb') as f:
    dfb = pickle.load(f)
    dfp = pickle.load(f)
    dff = pickle.load(f)
    teams = pickle.load(f)

To gather our totals for each team we take only the total row for each team and rename the dataframe `pitch_totals`. We also delete the `GS` column as the number of games started is not applicable for team totals, only individuals.

In [3]:
pitch_totals = [df[df.Pitching.str.contains("Total:", regex = False)] for df in dfp]
for df in pitch_totals:
    del df["GS"]
pitch_totals[2]

,Pitching,ERA,W,L,GP,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
12,Total:,4.46,6,10,16,2,0,0,1,127.0,136,90,63,45,104,13,4,4,555,0.275,13,10,2,4,1


We combine these total rows into a single dataframe named `merged_pitch_totals`.

In [4]:
merged_pitch_totals = pd.concat(pitch_totals)
merged_pitch_totals[:4]

,Pitching,ERA,W,L,GP,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
13,Total:,4.90,3,13,16,4,1,0,1,125.0,127,102,68,53,85,26,1,12,574,0.257,14,16,0,5,6
17,Total:,6.83,0,16,16,2,0,0,0,118.2,144,143,90,78,107,30,6,10,630,0.289,20,34,4,8,12
12,Total:,4.46,6,10,16,2,0,0,1,127.0,136,90,63,45,104,13,4,4,555,0.275,13,10,2,4,1
9,Total:,3.52,11,5,16,3,0,0,3,127.2,117,55,50,55,102,13,0,10,555,0.243,12,13,3,1,4


For those familiar with baseball, the interpretation of the `IP` column displaying the number of innings pitched by each player is fairly straightforward. However, the decimals are not mathematically equivalent to their numerical value. Because each inning consists of three outs, if a pitcher gets only one out in the inning, their `IP` total would display `0.1` even though mathematically, they threw 1/3 or 0.333... of an inning. To do our calculations we must fix this before summing the total innings pitched. First, we gather the innings pitched in an individual list named `ips`.

In [5]:
ips = merged_pitch_totals["IP"].to_list()

We then run a test for each number in our list. We multiply the number of innings by 10 to eliminate the decimal and then compare the modulus 10 of each number. If that number mod 10 is equal to 0, we know the innings pitched is a whole number and we do nothing. If the number mod 10 is equal to 1, we know that one extra out was earned and we correct the number to 0.333 rather than 0.1 by adding 0.233. Similarly, if the number mod 10 is equal to 2, we must add 0.467 to acheive a decimal of 0.667. Once we have changed each our innings, we copy the dataframe again to avoid any errors and reset the `IP` column to the values in the list `ips`.

In [6]:
for i in range(len(ips)):
    if ((ips[i]*10) % 10) == 0:
        pass
    elif ((ips[i]*10) % 10) == 1:
        ips[i] = ips[i] + 0.233
    elif ((ips[i]*10) % 10) == 2:
        ips[i] = ips[i] + 0.467
fixed_ip_totals = merged_pitch_totals.copy()
fixed_ip_totals["IP"] = ips
fixed_ip_totals[:4]

,Pitching,ERA,W,L,GP,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
13,Total:,4.90,3,13,16,4,1,0,1,125.000,127,102,68,53,85,26,1,12,574,0.257,14,16,0,5,6
17,Total:,6.83,0,16,16,2,0,0,0,118.667,144,143,90,78,107,30,6,10,630,0.289,20,34,4,8,12
12,Total:,4.46,6,10,16,2,0,0,1,127.000,136,90,63,45,104,13,4,4,555,0.275,13,10,2,4,1
9,Total:,3.52,11,5,16,3,0,0,3,127.667,117,55,50,55,102,13,0,10,555,0.243,12,13,3,1,4


We add a total row at the bottom with the code `.sum()` and we delete the `Pitching` column as it does not hold any meaningful information. Finally, we subset only the last column that contains the final totals with the code `.iloc[-1:]`.

In [7]:
fixed_ip_totals.loc["CL_Total"] = fixed_ip_totals.sum()
del fixed_ip_totals["Pitching"]
CL_pitch_totals = fixed_ip_totals.iloc[-1,:]

Since some of the statistics are averages, we have to go back and calculate those by hand because the summation incorrectly calculates those metrics. A few of the conference totals are printed.

In [8]:
CL_tot_p = CL_pitch_totals.copy()
#fix opponent batting average
CL_tot_p["B_AVG"] = round(CL_tot_p["H"] / (CL_tot_p["TBF"] - CL_tot_p["BB"] - CL_tot_p["HBP"] - CL_tot_p["SFA"] - CL_tot_p["SHA"]), 3)
#fix ERA
CL_tot_p["ERA"] = round((CL_tot_p["ER"]*27) / round(CL_tot_p["IP"]*3), 3)

In [9]:
CL_tot_p[:5]

ERA      4.609
W       80.000
L       79.000
GP     160.000
CG      21.000
Name: CL_Total, dtype: float64

Next, in order to rank the Crossroads League pitchers, we will calculate Fielding Independent Pitching (FIP), a statistic that measures a pitcher's effectiveness regardless of defense or inherent luck in hitting. Our first step is to calculate the league constant, which is shown below.

In [10]:
CL_FIP_c = CL_tot_p["ERA"] - (((13*CL_tot_p["HR"] + 3*(CL_tot_p["BB"] + CL_tot_p["HBP"]) - 2*CL_tot_p["SO"])*3) / round(CL_tot_p["IP"]*3))
CL_FIP_c

3.7692254428341387

Just like we did with the totals dataframe, we must correct for the notation used in the `IP` column for each team's individual statistical table.

In [11]:
for df in dfp:
    ips = df["IP"].to_list()
    for i in range(len(df.index)):
        if ((ips[i]*10) % 10) == 0:
            pass
        elif ((ips[i]*10) % 10) == 1:
            ips[i] = ips[i] + 0.233
        elif ((ips[i]*10) % 10) == 2:
            ips[i] = ips[i] + 0.467
    df["IP"] = ips

Now that we have workable data, we can calculate our individual FIP statistics for each pitcher. We also add a column for each player's team to improve readability in our final rankings. After creating the column, we move it up to be the first statistical column displayed.

In [12]:
for i in range(len(teams)): #add column for team
    dfp[i]["Team"] = teams[i]
for df in dfp:
    df["FIP"] = round(((13*df["HR"] + 3*(df["BB"] + df["HBP"]) - 2*df["SO"])*3) / round(df["IP"]*3) + CL_FIP_c, 3)
    team = df.pop("Team")
    df.insert(1, team.name, team) #move team column to second
    fip = df.pop("FIP")
    df.insert(2, fip.name, fip) #move FIP column to third

We tidy up the data by removing the totals and the opponents rows and removing the unnecessary decimal place in the `GS` column.

In [15]:
for df in dfp:
    #df.drop(df.tail(2).index,inplace=True) # drop last 2 rows(only run this code once otherwise data will be lost)
    df['GS'] = df['GS'].astype(int) #remove decimal place on GS column
dfp[2][-2:]

,Pitching,Team,FIP,ERA,W,L,GP,GS,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
6,"Decker, Jake",Grace,5.269,4.05,0,0,4,0,0,0,0,0,13.333,12,6,6,6,12,1,3,2,53,0.261,0,0,0,1,0
8,"Meyer, Mason",Grace,4.258,5.02,1,2,4,2,0,0,0,0,14.333,17,14,8,8,10,0,0,0,67,0.293,2,1,0,0,0


We subset the data to only include pitchers with at least 10 conference innings to ensure the best pitchers are recognized.

In [16]:
#make sure everyone has at least 10 IP
for i in range(len(teams)):
    dfp[i] = dfp[i][dfp[i]['IP'] >= 10]

Finally, we combine all of the dataframes with the code `pd.concat`. Then, we sort the rows by FIP beginning with the lowest. To maintain the standard notation of the `IP` column, we run a test to check for pitchers with innings pitched ending in 0.333 or 0.667 and change them back to 0.1 and 0.2 respectively. The top ten Crossroads League pitchers in terms of FIP are shown below.

In [17]:
all_pitchers = pd.concat(dfp)
top_pitchers = all_pitchers.sort_values(by=['FIP']) #sort by FIP in ascending order order
ips = top_pitchers["IP"].to_list() #change IP back to standard notation
for i in range(len(top_pitchers.index)):
    if ((ips[i]*3) % 3) == 0:
        pass
    elif (round(ips[i]*3) % 3) == 1:
        ips[i] = ips[i] - 0.233
    elif (round(ips[i]*3) % 3) == 2:
        ips[i] = ips[i] - 0.467
top_pitchers["IP"] = ips
top_pitchers[:10]

,Pitching,Team,FIP,ERA,W,L,GP,GS,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
6,"Gongwer, Drake",Taylor,1.717,1.42,1,0,4,0,0,0,0,0,12.2,4,2,2,1,19,2,0,0,45,0.100,1,3,1,0,0
10,"Moran, Joe",Taylor,1.978,2.63,2,0,4,4,0,0,1,0,24.0,21,7,7,7,32,4,0,0,100,0.233,2,0,0,1,2
11,"Ubelhor, Mitch",Taylor,2.269,2.84,0,0,5,0,0,0,1,0,12.2,6,4,4,7,20,2,0,0,51,0.136,3,0,0,0,0
4,"Hoffman, Hunter",IWU,2.769,1.13,1,0,4,4,0,0,0,0,24.0,13,6,3,12,30,1,0,0,97,0.153,1,0,0,0,0
7,"Huseman, Noah",Taylor,2.843,1.99,3,1,4,4,1,1,0,0,22.2,15,9,5,11,30,1,0,0,96,0.185,2,2,1,0,2
5,"Haney, Houston",Grace,2.876,3.54,1,2,4,4,1,0,0,0,28.0,30,13,11,2,20,3,0,0,113,0.283,0,3,0,1,1
1,"Shinabery, Mason",HU,3.055,1.61,4,0,4,4,2,0,0,0,28.0,17,5,5,6,27,5,0,1,105,0.173,3,1,0,0,0
12,"Cassel, Sean",SAU,3.144,6.19,2,1,4,4,0,0,0,0,16.0,17,13,11,11,23,2,2,0,78,0.262,1,1,0,0,1
7,"Norman, Braedon",IWU,3.406,2.45,2,0,4,1,0,0,1,0,11.0,8,3,3,3,8,3,0,0,44,0.200,2,1,0,0,0
9,"Shively, Luke",Taylor,3.614,2.33,2,0,4,4,1,1,0,0,19.1,12,6,5,6,20,3,1,1,81,0.164,1,2,0,0,0
